In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier as DC
from sklearn.ensemble import BaggingClassifier as BC
from sklearn.ensemble import RandomForestClassifier as RC
from sklearn.metrics import accuracy_score as acc
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score as cs
from sklearn.ensemble import GradientBoostingClassifier as GC
from xgboost import XGBClassifier as XGBS
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
import warnings
warnings.filterwarnings("ignore")


https://archive.ics.uci.edu/ml/datasets/wine+quality

In [2]:
link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
data = pd.read_csv(link, sep=';')
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


1.Оцените качество по метрике accuracy для классификаторов:

DecisionTreeClassifier

BaggingClassifier со 100 деревьями

RandomForestClassifier со 100 деревьями

Сравните результаты и напишите какой вывод можно сделать.

In [3]:
X = data[data.columns[:-1]]
y = data[data.columns[-1:]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
DC_tree = acc(y_test, DC().fit(X_train, y_train).predict(X_test))
BC_tree = acc(y_test, BC(n_estimators=100).fit(X_train, y_train).predict(X_test))
RC_tree = acc(y_test, RC(n_estimators=100).fit(X_train, y_train).predict(X_test))
print(DC_tree)
print(BC_tree)
print(RC_tree)


0.565625
0.634375
0.69375


2.Разделите выборку на обучающую и тестовую в отношении 70%/30%

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


3.Посчитайте качество на тестовой выборке по метрике accuracy для классификатора RandomForestClassifier, используя значения деревьев:
    
10, 50, 100, 200, далее с шагом 200 до 5000 деревьев.
Постройте график зависимости качества от числа деревьев.



In [6]:
tree = [10,50,100,200]
step = [i for i in range(400,5200,200)]
tree_step = tree + step
tree_step

[10,
 50,
 100,
 200,
 400,
 600,
 800,
 1000,
 1200,
 1400,
 1600,
 1800,
 2000,
 2200,
 2400,
 2600,
 2800,
 3000,
 3200,
 3400,
 3600,
 3800,
 4000,
 4200,
 4400,
 4600,
 4800,
 5000]

4.Обучите реализации градиентного бустинга с параметрами по умолчанию из библиотек sklearn и xgboost. Сравните значение метрики accuracy по cross_val_score

In [7]:
cs(GC(), X, y, scoring='accuracy', cv=5).mean()


0.5647198275862069

6.Обучите реализации градиентного бустинга с параметрами по умолчанию из библиотек lightgbm и catboost. Сравните значение метрики accuracy по cross_val_score по всем четырем реализациям.

In [8]:
cs(LGBMClassifier(), X, y, scoring='accuracy').mean()


0.5572472570532915

In [9]:
cs(CatBoostClassifier(), X, y, scoring='accuracy').mean()

Learning rate set to 0.080126
0:	learn: 1.6829142	total: 225ms	remaining: 3m 44s
1:	learn: 1.5940547	total: 258ms	remaining: 2m 8s
2:	learn: 1.5175367	total: 300ms	remaining: 1m 39s
3:	learn: 1.4573818	total: 318ms	remaining: 1m 19s
4:	learn: 1.3986323	total: 344ms	remaining: 1m 8s
5:	learn: 1.3493535	total: 368ms	remaining: 1m 1s
6:	learn: 1.3018142	total: 387ms	remaining: 54.9s
7:	learn: 1.2635278	total: 406ms	remaining: 50.4s
8:	learn: 1.2288350	total: 426ms	remaining: 46.9s
9:	learn: 1.1965165	total: 453ms	remaining: 44.8s
10:	learn: 1.1667856	total: 486ms	remaining: 43.7s
11:	learn: 1.1415420	total: 513ms	remaining: 42.3s
12:	learn: 1.1189592	total: 534ms	remaining: 40.5s
13:	learn: 1.0971406	total: 553ms	remaining: 38.9s
14:	learn: 1.0774057	total: 569ms	remaining: 37.4s
15:	learn: 1.0589907	total: 588ms	remaining: 36.1s
16:	learn: 1.0444871	total: 608ms	remaining: 35.2s
17:	learn: 1.0287948	total: 629ms	remaining: 34.3s
18:	learn: 1.0152259	total: 656ms	remaining: 33.9s
19:	lear

0.5459757053291535

Лучший результат у CatBoostClassifier

7.Подберите оптимальные параметры для алгоритмов градиентного бустинга из библиотек lightgbm и catboost с теми же условиями. Выведите лучшие параметры алгоритмов.
Сравните значение метрики accuracy и скорость по этим четырем реализациям.

In [10]:
par = {'learning_rate' : [0.01, 0.05, 0.1],
       'n_estimators' : [1, 2, 4, 8, 16, 32],
        'max_depth' : range(1,5,10)}

acc(y_test, GridSearchCV(LGBMClassifier(), par).fit(X_train, y_train).predict(X_test))


0.53125

In [11]:
acc(y_test, GridSearchCV(CatBoostClassifier(), par).fit(X_train, y_train).predict(X_test))

0:	learn: 1.7806349	total: 5.53ms	remaining: 0us
0:	learn: 1.7788956	total: 4ms	remaining: 0us
0:	learn: 1.7793570	total: 2.01ms	remaining: 0us
0:	learn: 1.7767511	total: 2.81ms	remaining: 0us
0:	learn: 1.7806800	total: 2.02ms	remaining: 0us
0:	learn: 1.7806349	total: 2.63ms	remaining: 2.63ms
1:	learn: 1.7663641	total: 4.71ms	remaining: 0us
0:	learn: 1.7788956	total: 2.88ms	remaining: 2.88ms
1:	learn: 1.7642995	total: 8.39ms	remaining: 0us
0:	learn: 1.7793570	total: 2.03ms	remaining: 2.03ms
1:	learn: 1.7649599	total: 4.21ms	remaining: 0us
0:	learn: 1.7767511	total: 2.14ms	remaining: 2.14ms
1:	learn: 1.7628564	total: 4.29ms	remaining: 0us
0:	learn: 1.7806800	total: 2.09ms	remaining: 2.09ms
1:	learn: 1.7699031	total: 4.24ms	remaining: 0us
0:	learn: 1.7806349	total: 4.71ms	remaining: 14.1ms
1:	learn: 1.7663641	total: 9.7ms	remaining: 9.7ms
2:	learn: 1.7523680	total: 11.2ms	remaining: 3.73ms
3:	learn: 1.7387791	total: 13.1ms	remaining: 0us
0:	learn: 1.7788956	total: 4.3ms	remaining: 12.9ms

0.5520833333333334

9.Выведите качество по метрике accuracy стэкинга (StackingClassifier) 4-х алгоритмов с базовыми параметрами градиентного бустинга.

In [ ]:
date = [('GC', GC()),
        ('LGBM', LGBMClassifier()),
        ('XGBC', XGBS())]
date_stack = StackingClassifier(estimators=date, final_estimator=CatBoostClassifier())
date_stack.fit(X_train, y_train)

In [13]:
acc(np.ravel(y_test, ), date_stack.predict(X_test))

0.63125

10.Выведите качество по метрике accuracy стэкинга 4-х алгоритмов с оптимальными параметрами градиентного бустинга. Сравните результаты с предыдущим шагом и напишите какой вывод можно из этого сделать.

In [ ]:
date = [('GC', GC(learning_rate=0.01, max_depth=2, n_estimators= 10)),
        ('LGBM', LGBMClassifier(learning_rate=0.02, max_depth=3, n_estimators= 20)),
        ('XGBC', XGBS(learning_rate=0.03, max_depth=4, n_estimators= 30))]
date_stack = StackingClassifier(estimators=date, final_estimator=CatBoostClassifier(learning_rate=0.01, max_depth=2, n_estimators= 20))
date_stack.fit(X_train, y_train)

In [15]:
acc(np.ravel(y_test, ), date_stack.predict(X_test))

0.5416666666666666

Лучшее качество показывает стекинг с параметрами.